**SETUP: Virtual Environment Setup (for Python 3.10)**

1. **Install Python 3.10 if you haven't already (check your Python version with `python --version` or `python3 --version`).**
   - Download Python 3.10 from: https://www.python.org/downloads/release/python-3100/

2. **Create a virtual environment with Python 3.10**:
    ```bash
    python3.10 -m venv venv
    ```

3. **Activate the virtual environment**:
    - **Windows**:
      ```bash
      venv\Scripts\activate
      ```
    - **macOS/Linux**:
      ```bash
      source venv/bin/activate
      ```

4. **Install the dependencies from the `requirements.txt` file**:
    ```bash
    pip install -r requirements.txt
    ```

5. **Install the Jupyter kernel for the virtual environment**:

   ```bash
   python -m ipykernel install --user --name=venv --display-name "Python 3.10 (venv)"
   ```

6. **Deactivate the virtual environment when done**:
    ```bash
    deactivate
    ```


In [5]:
# Cell 1: Import libraries

import tensorflow as tf
from keras.api.models import Sequential
from keras.api.layers import Dense, Dropout, GlobalAveragePooling2D
from keras.api.callbacks import EarlyStopping, ReduceLROnPlateau
from keras.api.preprocessing import image_dataset_from_directory
from keras.api.applications import ResNet50
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, classification_report
import seaborn as sns
from sklearn.utils.class_weight import compute_class_weight
import numpy as np

In [6]:
# Cell 2: Set Paths and Parameters

# Paths to dataset
TRAINING_DIR = '../chest_xray/train'
VALIDATION_DIR = '../chest_xray/val'
TEST_DIR = '../chest_xray/test'

# Parameters
IMG_SIZE = 224  # Resizing
BATCH_SIZE = 32
EPOCHS = 30

In [ ]:
# Cell 3: Load and Preprocess the Dataset

train_dataset = image_dataset_from_directory(
    TRAINING_DIR,
    image_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    label_mode='binary',
    seed=123
)

val_dataset = image_dataset_from_directory(
    VALIDATION_DIR,
    image_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    label_mode='binary',
    seed=123
)

test_dataset = image_dataset_from_directory(
    TEST_DIR,
    image_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    label_mode='binary',
    seed=123
)

# Configure datasets for performance
AUTOTUNE = tf.data.AUTOTUNE
train_dataset = train_dataset.cache().prefetch(buffer_size=AUTOTUNE)
val_dataset = val_dataset.cache().prefetch(buffer_size=AUTOTUNE)
test_dataset = test_dataset.cache().prefetch(buffer_size=AUTOTUNE)

In [8]:
# Cell 4: Extract Labels and Compute Class Weights

# Extract labels from the training dataset
train_labels = np.concatenate([y.numpy() for _, y in train_dataset])
train_labels = train_labels.flatten()  # Ensure the labels are in a flat 1D array

# Compute class weights based on the labels
class_weights = compute_class_weight(
    class_weight='balanced',
    classes=np.unique(train_labels),
    y=train_labels
)

# Create a dictionary of class weights
class_weights_dict = {i: class_weights[i] for i in range(len(class_weights))}

In [9]:
# Cell 5: Data Augmentation Layer

# Enhanced Data Augmentation Layer using tf.keras.layers
data_augmentation = tf.keras.Sequential([
    tf.keras.layers.RandomRotation(0.25),
    tf.keras.layers.RandomZoom(0.25),
    tf.keras.layers.RandomWidth(0.2),
    tf.keras.layers.RandomHeight(0.2),
    tf.keras.layers.RandomContrast(0.3),
    tf.keras.layers.RandomBrightness(0.2),
])

In [10]:
# Cell 6: Model Architecture with ResNet50

# Transfer learning with ResNet50
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(IMG_SIZE, IMG_SIZE, 3))
base_model.trainable = False  # Freeze the base model layers initially

# Model Architecture
model = Sequential([
    data_augmentation,                                                                  # Data Augmentation Layer
    base_model,                                                                         # Pretrained ResNet50
    GlobalAveragePooling2D(),                                                           # Global Average Pooling to reduce dimensions
    Dense(1024, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.02)),  # Dense with L2 regularization
    Dropout(0.7),                                                                       # Dropout rate set to 0.7 for regularization
    Dense(1, activation='sigmoid')                                                      # Output layer for binary classification
])

# Compile the model with an initial learning rate
model.compile(
    loss='binary_crossentropy',
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),
    metrics=['accuracy']
)

In [11]:
# Cell 7: Callbacks for Early Stopping and Learning Rate Scheduling

# Early stopping and ReduceLROnPlateau to help stabilize the training
early_stopping = EarlyStopping(monitor='val_loss', patience=6, restore_best_weights=True)
lr_scheduler = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=4, min_lr=1e-6)

In [ ]:
# Cell 8: Model Training

history = model.fit(
    train_dataset,
    epochs=EPOCHS,
    validation_data=val_dataset,
    class_weight=class_weights_dict,         # Explicitly use class weights
    callbacks=[early_stopping, lr_scheduler]
)

In [13]:
# Cell 9: Fine-Tuning the Model

# Unfreeze the base model layers after initial training
base_model.trainable = True  # Unfreeze all layers of the base model

# Fine-tune more layers
for layer in base_model.layers[:100]:  # Freeze the first 100 layers
    layer.trainable = False

In [14]:
# Cell 10: Custom Cyclical Learning Rate Scheduler

# Define the custom cyclical learning rate schedule
class CyclicalLearningRate(tf.keras.optimizers.schedules.LearningRateSchedule):
    def __init__(self, initial_lr, maximal_lr, step_size):
        self.initial_lr = initial_lr
        self.maximal_lr = maximal_lr
        self.step_size = step_size

    def __call__(self, step):
        cycle = tf.floor(1 + step / (2 * self.step_size))
        x = tf.abs(step / self.step_size - 2 * cycle + 1)
        lr = self.initial_lr + (self.maximal_lr - self.initial_lr) * tf.maximum(0., (1 - x))
        return lr
    
    def get_config(self):
        return {
            "initial_lr": self.initial_lr,
            "maximal_lr": self.maximal_lr,
            "step_size": self.step_size
        }

# Set your desired values for initial learning rate, maximal learning rate, and step size
initial_lr = 1e-5
maximal_lr = 1e-4
step_size = 500  # Number of steps per cycle

# Instantiate the cyclical learning rate schedule
lr_schedule = CyclicalLearningRate(initial_lr, maximal_lr, step_size)

# Compile the model with the cyclical learning rate schedule
model.compile(
    loss='binary_crossentropy',
    optimizer=tf.keras.optimizers.Adam(learning_rate=lr_schedule),
    metrics=['accuracy']
)

In [ ]:
# Cell 11: Continue Training with Fine-Tuning

# Continue training for more epochs with fine-tuning
history_finetune = model.fit(
    train_dataset,
    epochs=15,
    validation_data=val_dataset,
    class_weight=class_weights_dict,
    callbacks=[early_stopping]
)

In [16]:
# Cell 12: Save the Fine-Tuned Model

model.save('pneumonia_detection_model_finetuned.keras')

In [ ]:
# Cell 13: Plot Training and Validation History

# Accuracy and loss graphs
def plot_history(history):
    plt.figure(figsize=(14, 7))

    plt.subplot(1, 2, 1)
    plt.plot(history.history['accuracy'], label='Training Accuracy')
    plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
    plt.title('Accuracy Over Epochs')
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    plt.legend()

    plt.subplot(1, 2, 2)
    plt.plot(history.history['loss'], label='Training Loss')
    plt.plot(history.history['val_loss'], label='Validation Loss')
    plt.title('Loss Over Epochs')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()

    plt.show()

plot_history(history_finetune)

In [18]:
# Cell 14: Prediction Function

# Function to predict all images in the dataset
def predict_all_images(dataset):
    true_labels = []
    pred_labels = []

    for images, labels in dataset:
        preds = model.predict(images)
        true_labels.extend(labels.numpy())
        pred_labels.extend((preds > 0.5).astype(int))

    return true_labels, pred_labels

In [19]:
# Cell 15: Plot Confusion Matrix

def plot_confusion_matrix(true_labels, pred_labels):
    cm = confusion_matrix(true_labels, pred_labels)
    plt.figure(figsize=(7, 6))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=['Normal', 'Pneumonia'], yticklabels=['Normal', 'Pneumonia'])
    plt.xlabel('Predicted Label')
    plt.ylabel('True Label')
    plt.title('Confusion Matrix')
    plt.show()

In [ ]:
# Cell 16: Validate and Test the Model

true_labels, pred_labels = predict_all_images(val_dataset)
print("Validation Set Results:")
print(classification_report(true_labels, pred_labels, target_names=['Normal', 'Pneumonia']))
plot_confusion_matrix(true_labels, pred_labels)

true_labels_test, pred_labels_test = predict_all_images(test_dataset)
print("Test Set Results:")
print(classification_report(true_labels_test, pred_labels_test, target_names=['Normal', 'Pneumonia']))
plot_confusion_matrix(true_labels_test, pred_labels_test)